In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2'
sys.path.insert(0,my_path + r'/code/COMMON')

# imports
import numpy as np 
import matplotlib.pyplot as plt
import pickle

# Build features through Matrix Factorization

In [2]:
import os
os.chdir('D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2\data')

In [ ]:
os.system('build_vocab.sh')
os.system('cut_vocab.sh')

In [ ]:
from pickle_vocab import main as pickle_vocab
from cooc import main as cooc

pickle_vocab()
cooc()# cooccurrence matrix

In [ ]:
from glove_solution import main as glove

glove()

In [3]:
X = np.load('embeddings.npy')
print(X.shape)

(21750, 20)


In [4]:
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [5]:
remove_list = ['<user>', ',']

In [6]:
# build positive tweet feature set
X_pos = []

with open('train_pos.txt') as f:
    
    for line in f:
        
        tweet = line.lstrip().split()
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_pos.append(list(np.mean(X_tweet, 0)))
        
    X_pos = np.array(X_pos)

In [7]:
# build negative tweet feature set
X_neg = []

with open('train_neg.txt') as f:
    
    for line in f:
        
        tweet = line.lstrip().split()
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_neg.append(list(np.mean(X_tweet, 0)))
        
    X_neg = np.array(X_neg)     

In [8]:
y_pos = np.ones(X_pos.shape[0])
y_neg = -np.ones(X_neg.shape[0])

In [9]:
print(X_pos.shape)
print(X_neg.shape)

(99952, 20)
(99989, 20)


In [10]:
X_pos_neg = np.concatenate([X_pos, X_neg])
y_pos_neg = np.concatenate([y_pos, y_neg])

# Training and testing sets

In [ ]:
ind = np.arange(y_pos_neg.shape[0])
np.random.shuffle(ind)

r = 0.7
ir = round(r*X_pos_neg.shape[0])

X_tr = X_pos_neg[ind[:ir],:]
X_te = X_pos_neg[ind[ir:],:]
y_tr = y_pos_neg[ind[:ir]]
y_te = y_pos_neg[ind[ir:]]

# SVM 

In [11]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_pos_neg, y_pos_neg) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Submission

In [ ]:
X_test = []
ids_test = []

with open('my_test.txt') as f:
    
    for line in f:
        print(line)
        id_ = line.lstrip().split(',')[0]
        ids_test.append(id_)
        tweet = line.lstrip().split(',')[1]
        print(id_)
        print(tweet)
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_test.append(list(np.mean(X_tweet, 0)))
        
X_test = np.array(X_test)
ids_test = np.array(ids_test)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from create_csv_submission import create_csv_submission

name = 'sub512'
create_csv_submission(ids_test, y_pred, name)